## Introduction to CUDA

### Learning objectives

This lab serves as an introduction to NVIDIA's accelerated computing platform, CUDA. The CUDA platform provides a general application interface for parallel computing on [CUDA-enabled GPUs](https://developer.nvidia.com/cuda-gpus). CUDA can be utilized from many programming languages, including C/C++, Fortran, and Python.

In this lab you will learn:

- The overall CUDA architecture, and the concept of accelerated computing
- CUDA C++, an extension to standard C++ that enables GPU accelerated computing
- How to accelerate common operations such as for loops on GPUs with CUDA
- How to allocate and work with memory on GPUs

After completing this lab, you will be able to take a standard serial C++ code, convert it into a parallel CUDA program, and possibly see a very significant speedup by running the program on an NVIDIA GPU. The topics we cover in this course are also applicable to other CUDA-enabled languages.

### Prerequisites

No GPU knowledge is assumed for this lab. Basic knowledge of C/C++ is assumed.

## Parallel Computing in C++

GPU computing is about massive parallelism. We'll need an appropriate example to get us started. Let's go with vector addition. We'll learn some semantic details about how to write CUDA on a toy "hello world" application and then implement a vector addition example.

![](images/vector_addition.png)

### GPU Kernels: Device Code

<br />
<span style="font-family:courier;font-size:2em">
<span style="color:orange">__global__</span> <span style="color:green">void</span> mykernel() { <br />}
</span>

The CUDA C++ keyword <span style="font-family:courier;color:orange">\_\_global\_\_</span> indicates a function that

- Runs on the device
- Is called from host code (can also be called from other device code)

The execution of the function on the device happens as a "kernel," and the function itself is often commonly referred to as a "kernel."

`nvcc`, the NVIDIA CUDA C++ compiler driver, separates source code into host and device components
- Device functions (e.g. `mykernel()`) processed by NVIDIA compiler
- Host functions (e.g. `main()`) processed by standard host compiler (e.g. gcc, icc, msvc)

<br />
<span style="font-family:courier;font-size:2em">
mykernel<span style="color:orange"><<<</span>1,1<span style="color:orange">>>></span>();
</span>

Triple angle brackets mark a call to *device* code.
- Also called a “kernel launch”
- We’ll return to the parameters (1,1) in a moment
  - For now, we'll say that these parameters inside the triple angle brackets are the CUDA kernel **execution configuration**

That’s all that is required to execute a function on the GPU!

## Exercise

Let's start testing out our knowledge. Edit the file [exercises/hello.cu](exercises/hello.cu) (by clicking the link to open it in a new tab, making edits directly in the source, and then using `File > Save` to save your changes) so that when it runs, it prints out the following:

```
Hello world
```

(Look for `FIXME` in the code.) 

Note the use of `cudaDeviceSynchronize()` after the kernel launch. In CUDA, kernel launches are asynchronous with respect to the host thread. The host thread will launch a kernel but not wait for it to finish, before proceeding with the next line of host code. Therefore, to prevent application termination before the kernel gets to print out its message, we must use this synchronization function.

Once you've updated the code, compile and run it and make sure your output matches what we expect.

In [1]:
!nvcc -arch=native -o hello exercises/hello.cu; ./hello

Hello world


If you get stuck, look at [solutions/hello.cu](solutions/hello.cu).

### Running Code in Parallel

GPU computing is about massive parallelism. So how do we run code in parallel on the device?

Instead of executing `hello()` once, we can execute it `N` times in parallel by taking

<span style="font-family:courier;font-size:large">hello<<<<span style="color:orange">1</span>,1>>>();</span>

and turning it into

<span style="font-family:courier;font-size:large">hello<<<<span style="color:orange">N</span>,1>>>();</span>

The parameter <span style="font-family:courier;color:orange">N</span> here means we want to execute the body of the kernel  <span style="font-family:courier;color:orange">N</span> times in parallel. Each execution of the kernel function `hello` is done by a so-called "thread block" or simply "block" (we will understand this terminology better later). The set of all thread blocks executing the kernel function is called a "grid." We can programmatically obtain the index of which thread block is currently executing the function using the CUDA-provided runtime variable `blockIdx.x` (which is zero indexed and runs from 0 to N-1). That is, within a kernel we can write:

```
__global__ void kernel() {

    int my_block = blockIdx.x;

}
```

and one execution of the kernel will have `my_block == 0`, another will have `my_block == 1`, etc., all the way up to the instance that has `my_block == N-1`.

## Exercise

Let's now test what we know about running code in parallel. Edit the file [exercises/hello_with_blocks.cu](exercises/hello_with_blocks.cu) so that when it runs, it prints out the following:

```
Hello from block: 0
Hello from block: 1
```

(Look for `FIXME` in the code.) 

Note that the ordering of the above lines may vary; ordering differences do not indicate an incorrect result.

Once you've updated the code, compile and run it and make sure your output matches what we expect.

In [2]:
!nvcc -arch=native -o hello_with_blocks exercises/hello_with_blocks.cu; ./hello_with_blocks

Hello from block: 7
Hello from block: 4
Hello from block: 1
Hello from block: 6
Hello from block: 3
Hello from block: 0
Hello from block: 5
Hello from block: 2


If you get stuck, look at [solutions/hello_with_blocks.cu](solutions/hello_with_blocks.cu).

## CUDA Threads

A thread block can be subdivided into parallel **threads**. If we change the *second* parameter in the execution configuration, this determines how many threads each thread block is subdivided into. That is, by taking

<span style="font-family:courier;font-size:large">hello<<<N,<span style="color:green">1</span>>>();</span>

and turning it into

<span style="font-family:courier;font-size:large">hello<<<N,<span style="color:green">M</span>>>>();</span>

the kernel `hello()` will now be launched with `N` thread blocks, each of which has `M` threads. Each thread gets its own invocation of the kernel, so the total number of invocations of the kernel body is now `N * M`. The new syntax we can use to identify which thread is currently executing is `threadIdx.x`:

```
__global__ void kernel() {

    int my_block = blockIdx.x;
    int my_thread_in_block = threadIdx.x;

}
```

We'll understand later why this two-level hierarchy exists. For now, we accept it as an aspect of the programming model that we must be comfortable with.

## Exercise

Let's practice with threads. Edit the file [exercises/hello_with_threads.cu](exercises/hello_with_threads.cu) so that when it runs, it prints out the following:

```
Hello from block: 0, thread: 0
Hello from block: 0, thread: 1
Hello from block: 1, thread: 0
Hello from block: 1, thread: 1
```

(Look for `FIXME` in the code.) 

As before, the ordering of the above lines may vary; ordering differences do not indicate an incorrect result.

Once you've updated the code, compile and run it and make sure your output matches what we expect.

In [3]:
!nvcc -arch=native -o hello_with_threads exercises/hello_with_threads.cu; ./hello_with_threads

Hello from block: 1, thread: 0
Hello from block: 1, thread: 1
Hello from block: 1, thread: 2
Hello from block: 1, thread: 3
Hello from block: 3, thread: 0
Hello from block: 3, thread: 1
Hello from block: 3, thread: 2
Hello from block: 3, thread: 3
Hello from block: 0, thread: 0
Hello from block: 0, thread: 1
Hello from block: 0, thread: 2
Hello from block: 0, thread: 3
Hello from block: 2, thread: 0
Hello from block: 2, thread: 1
Hello from block: 2, thread: 2
Hello from block: 2, thread: 3


If you get stuck, look at [solutions/hello_with_threads.cu](solutions/hello_with_threads.cu).

### Vector Addition on the Device

Now let's look at an example of a kernel that does parallel vector addition on arrays, $c = a + b$. We'll call it `add()` and it might look like:

<span style="font-family:courier;font-size:large">
<span style="color:green">__global__</span> void add(int* a, int* b, int* c) {<br />
    &nbsp; &nbsp; &nbsp; &nbsp; c[<span style="color:orange">blockIdx.x</span>] = a[<span style="color:orange">blockIdx.x</span>] + b[<span style="color:orange">blockIdx.x</span>];<br />
}<br /><br />
</span>


When we call this kernel from `main()`, we would use

```
    add<<<N, 1>>>(a, b, c);
```

By using **blockIdx.x** to index into the array, each block handles a different index of the arrays (and that block handles the same index for all three arrays).

Separately, we might want to also implement it with only threads:

<span style="font-family:courier;font-size:large">
<span style="color:green">__global__</span> void add(int* a, int* b, int* c) {<br />
    &nbsp; &nbsp; &nbsp; &nbsp; c[<span style="color:orange">threadIdx.x</span>] = a[<span style="color:orange">threadIdx.x</span>] + b[<span style="color:orange">threadIdx.x</span>];<br />
}<br /><br />
</span>

When we call this kernel from `main()`, we would use

```
    add<<<1, M>>>(a, b, c);
```

By using **threadIdx.x** to index into the array, every thread in the block handles a different index of the arrays (and that thread handles the same index for all three arrays).

If there's two ways to do this, which is preferred? It turns out that the best approach is to *combine* blocks and threads to solve problems. (We'll understand this better when we learn more about the GPU architecture.)

## Combining Blocks and Threads

We’ve seen parallel vector addition using:
- Many blocks with one thread each
- One block with many threads

Let’s adapt vector addition to use both blocks and threads, with a focus on how to index the data.

### Indexing Arrays with Blocks and Threads

This is no longer as simple as using <span style="font-family:courier;">**blockIdx.x**</span> and <span style="font-family:courier;">**threadIdx.x**</span>. Consider indexing an array with one element per thread (8 threads/block):

![](images/block_and_thread_indexing.png)

With `M` threads/block a unique index for each thread is given by:

```
int index = threadIdx.x + blockIdx.x * M;
```

### Indexing Arrays: Example

Which thread will operate on the red element?

![](images/indexing_arrays_example.png)

### Vector Addition with Blocks and Threads

Use the built-in variable <span style="font-family:courier;color:orange">**blockDim.x**</span> to get the number of threads per block.

<span style="font-family:courier;font-size:large">
    int index = threadIdx.x + blockIdx.x * <span style="color:orange">blockDim.x</span>;<br /><br />
</span>

Now we can write a combined version of `add()` to use parallel threads and parallel blocks:

<span style="font-family:courier;font-size:large">
<span style="color:green">__global__</span> void add(int* a, int* b, int* c) {<br />
    &nbsp; &nbsp; &nbsp; &nbsp; int index = threadIdx.x + blockIdx.x * blockDim.x;<br/>
    &nbsp; &nbsp; &nbsp; &nbsp; c[<span style="color:orange">index</span>] = a[<span style="color:orange">index</span>] + b[<span style="color:orange">index</span>];<br />
}<br /><br />
</span>

and when we call this kernel from `main()` we need to do

```
    add<<<N / M, M>>>(a, b, c);
```

where `M` is the number of threads per block we choose, and `N` is the length of the arrays. `N / M` then ensures we launch as many threads as there are elements in the array (assuming it is an integer multiple of `M`).

## Exercise

Let's practice with combined thread and block indexing. Edit the file [exercises/hello_with_a_specific_thread.cu](exercises/hello_with_a_specific_thread.cu) so that when it runs, it prints out *only* (and *exactly once*) the following:

```
Hello from unique thread index: 773
```

(Look for `FIXME` in the code.) 

In [4]:
!nvcc -arch=native -o hello_with_a_specific_thread exercises/hello_with_a_specific_thread.cu; ./hello_with_a_specific_thread

Hello from block: 1, unique thread: 2


If you get stuck, look at [solutions/hello_with_a_specific_thread.cu](solutions/hello_with_a_specific_thread.cu). Note that this solution is not unique.

## Handling Arbitrary Vector Sizes

By now, we can see that the total number of threads in a kernel launch is an integer multiple of the number of threads in a block, but typical problems are not friendly multiples of <span style="font-family:courier;">**blockDim.x**</span>. We must avoid accessing beyond the end of the arrays:

<span style="font-family:courier;font-size:large">
<span style="color:green">__global__</span> void add(int* a, int* b, int* c, int N) {<br />
    &nbsp; &nbsp; &nbsp; &nbsp; int index = threadIdx.x + blockIdx.x * blockDim.x;<br/>
    &nbsp; &nbsp; &nbsp; &nbsp; if (index < N)<br/>
    &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; c[<span style="color:orange">index</span>] = a[<span style="color:orange">index</span>] + b[<span style="color:orange">index</span>];<br />
}<br /><br />
</span>

Note that we added the length of the arrays `N` as a kernel parameter. The updated kernel launch looks like:

<span style="font-family:courier;font-size:large">
    add<<<<span style="color:orange">(N + M - 1) / M</span>, M>>>(a, b, c, N);<br /><br />
</span>

`(N + M - 1) / M` is a simple trick for doing integer division that rounds upward (to ensure we have at least as many threads as there are elements in the array, with a few threads left over if needed).

Before we can tackle our vector addition example, we have to learn about one more topic.

### Simple Processing Flow

A typical accelerated program using CUDA employs the following workflow. We start with execution on the CPU, and copy our input data from the CPU memory (host memory) to the GPU memory (device memory).

![](images/simple_processing_flow_1.png)

Next, we load up the code for execution on the GPU and run the code on the GPU. The computations are done on the GPU, and the GPU can cache the data from its own memory in local on-chip caches for maximum performance.

![](images/simple_processing_flow_2.png)

Finally we copy the results back from GPU memory to CPU memory.

![](images/simple_processing_flow_3.png)

So let's learn about our tools for memory management.

### Memory Management

Host and device memory are separate entities:


- **Device** pointers point to GPU memory
  - Typically passed to device code
  - Typically *not* dereferenced in host code


- **Host** pointers point to CPU memory
  - Typically not passed to device code
  - Typically *not* dereferenced in device code
  
  
There are some special cases we won't cover here, including pinned memory and managed memory, and platforms with coherent CPU/GPU memory accesses such as IBM Power9 servers.

There is a simple CUDA API for handling device memory.

- `cudaMalloc()`, `cudaFree()`, `cudaMemcpy()`

These are analogous to standard CPU memory management functions in C

- `malloc()`, `free()`, `memcpy()`

The syntax of the allocation and deallocation APIs looks like:

```
    cudaMalloc(&ptr, size_in_bytes_to_allocate);
    cudaFree(ptr);
```

So `cudaFree` looks like `free`, but `cudaMalloc` is a little different from `malloc` in that it takes the address of the pointer and modifies it directly (this is because, as we're about to see, CUDA APIs generally return error codes so you know whether they worked, so the allocated pointer cannot also be the return value of the API).

The syntax of the memcpy API looks like:

```
    cudaMemcpy(destination_ptr, source_ptr, size_in_bytes_to_copy, direction);
```

where `direction` can be `cudaMemcpyHostToDevice` (for copying data from CPU to GPU) or `cudaMemcpyDeviceToHost` (for copying data from GPU to CPU).

## Exercise

Let's practice with memory allocation. In [exercises/memory_management.cu](exercises/memory_management.cu), allocate data for a single integer, set it to an initial value on the host, copy that initial value to the GPU, then launch a kernel to change its value, then copy the new value back to the host and verify that it changed as you expected. Consult [solutions/memory_management.cu](solutions/memory_management.cu) if you get stuck.

In [5]:
!nvcc -arch=native -o memory_management exercises/memory_management.cu; ./memory_management

Success!


## Error Handling

Every CUDA runtime API call returns an error code. It's good practice (especially if you're having trouble) to rigorously check these error codes. The error code is type `cudaError_t` (which is an integer). If the value of the error is `cudaSuccess` (0) then the CUDA API call returned without errors. Otherwise some error occurred, and CUDA provides a number of error codes corresponding to different failure conditions. See the [error handling](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__ERROR.html#group__CUDART__ERROR) section of the CUDA Runtime API for more information, and specifically you view the [cudaError enum](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html#group__CUDART__TYPES_1g3f51e3575c2178246db0a94a430e0038) for a list of the different error codes.

To use this in code, it's as simple as

```
    int* a;
    // Illegal: cannot allocate a negative number of bytes
    cudaError_t err = cudaMalloc(&a, -1);
    if (err != cudaSuccess) {
        printf("CUDA error %d\n", err);
        exit(-1);
    }
```

Since you won't have the error codes memorized by heart, CUDA provides a convenience function `cudaGetErrorString` for returning a human-readable string corresponding to a given error code.

```
    int* a;
    // Illegal: cannot allocate a negative number of bytes
    cudaError_t err = cudaMalloc(&a, -1);
    if (err != cudaSuccess) {
        printf("CUDA error %s\n", cudaGetErrorString(err));
        exit(-1);
    }
```

One additional issue to handle is that `__global__` functions have void return type; how do we get errors that occur in kernels? These errors can come in two flavors. One class of errors is errors in the kernel launch API (the triple chevron syntax is just syntactic sugar that maps to an actual CUDA runtime API call, `cudaLaunchKernel`). To detect any errors in the kernel launch (that is, an invalid parameter), we can use the API call `cudaGetLastError()` which returns the error code for whatever the last CUDA API call was. The second class of error is an error that occurs asynchronously during the kernel launch. `cudaGetLastError()` called immediately after the kernel will not generally pick this up (the error may not have even happened by the time it checks). But fortunately, if an asynchronous error has been detected, the next CUDA runtime API will catch it. So if we insert a `cudaDeviceSynchronize()` after the kernel, this call will return any errors associated with the kernel launch. 

```
    // Illegal: number of blocks cannot be negative
    kernel<<<-1,1>>>();
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA error %s\n", cudaGetErrorString(err));
        exit(-1);
    }
    
    err = cudaDeviceSynchronize();
    if (err != cudaSuccess) {
        printf("CUDA error %s\n", cudaGetErrorString(err));
        exit(-1);
    }
```

## Exercise

The code [exercises/error_handling.cu](exercises/error_handling.cu) intentionally has several mistakes. Try locating the errors yourself by inspection, then before fixing them insert CUDA error checking after all the CUDA runtime API calls and see if the results match your expectation. Then fix the errors one by one until the code runs to completion without errors. An example solution is provided in [solutions/error_handling.cu](solutions/error_handling.cu).

In [6]:
!nvcc -arch=native -o error_handling exercises/error_handling.cu; ./error_handling

exercises/error_handling.cu(19): warning #68-D: integer conversion resulted in a change of sign

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

exercises/error_handling.cu(19): warning #68-D: integer conversion resulted in a change of sign

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

Error: out of memory


## Exercise

Now let's pull together all the skills we've learned in this lab. We've given you a skeleton version of the vector addition program in [exercises/vector_addition.cu](exercises/vector_addition.cu). Edit the code to build a complete vector_add program. (Or, if you're up for a challenge, see if you can write the complete vector addition program from scratch.) Compile it and run it. You can refer to [solutions/vector_addition.cu](solutions/vector_addition.cu) for a complete example.

Note that this skeleton code includes a macro for CUDA error checking to make your life easier.

Typical output when complete would look like this:

```
A[0] = 0.840188
B[0] = 0.394383
C[0] = 1.234571
```

The important thing is that `C[0] = A[0] + B[0]` if you've obtained correct execution.

In [7]:
!nvcc -arch=native -o vector_addition exercises/vector_addition.cu; ./vector_addition

A[0] = 0.840188
B[0] = 0.394383
C[0] = 1.234571


## Review

In this lab we have learned:

- The difference between host (CPU) and device (GPU)


- Using `__global__` to declare a function as device code
  - Executes on the device
  - Called from the host (or possibly from other device code)


- Passing parameters from host code to a device function

 
- How to compile and run CUDA code


- Basic memory management (`cudaMalloc()`, `cudaFree()`, `cudaMemcpy`)


- Launching parallel kernels that use both blocks and threads
  - Launch `N` copies of `add()` with `add<<<N,1>>>(...);`
  - Use `blockIdx.x` to access block index
  - Use `threadIdx.x` to access thread index within block
  - Assign elements to threads: `int index = threadIdx.x + blockIdx.x * blockDim.x;`

## Further Study

[An introduction to CUDA](https://devblogs.nvidia.com/easy-introduction-cuda-c-and-c/)

[Another introduction to CUDA](https://devblogs.nvidia.com/even-easier-introduction-cuda/)

[CUDA Programming Guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html)

[CUDA Runtime API Documentation](https://docs.nvidia.com/cuda/index.htmlhttps://docs.nvidia.com/cuda/cuda-runtime-api/index.html)

## Optional Exercise

A skeleton code for a (naive) matrix multiply is given to you in [exercises/matrix_mul.cu](exercises/matrix_mul.cu). See if you can complete it to get a correct result. If you need help, you can refer to [solutions/matrix_mul.cu](solutions/matrix_mul.cu).

This example introduces 2D threadblock/grid indexing, something we did not previously cover. If you study the code you will probably be able to see how it is a structural extension from the 1D case.

This code includes built-in error checking, so a correct result is indicated by the program.

In [5]:
!nvcc -arch=native -o matrix_mul exercises/matrix_mul.cu; ./matrix_mul


Matrix A (Top-4x4 Corner):
  1.0000   1.0000   1.0000   1.0000 
  1.0000   1.0000   1.0000   1.0000 
  1.0000   1.0000   1.0000   1.0000 
  1.0000   1.0000   1.0000   1.0000 


Matrix B (Top-4x4 Corner):
  2.0000   2.0000   2.0000   2.0000 
  2.0000   2.0000   2.0000   2.0000 
  2.0000   2.0000   2.0000   2.0000 
  2.0000   2.0000   2.0000   2.0000 

Init took 0.081411 seconds.  Begin compute
Launching kernel with grid=(256,256) and block=(16,16)

Result Matrix C (Top-4x4 Corner):
8192.0000 8192.0000 8192.0000 8192.0000 
8192.0000 8192.0000 8192.0000 8192.0000 
8192.0000 8192.0000 8192.0000 8192.0000 
8192.0000 8192.0000 8192.0000 8192.0000 

Done. Compute took 0.477433 seconds
Success!

Result Matrix C (Top-4x4 Corner):
8192.0000 8192.0000 8192.0000 8192.0000 
8192.0000 8192.0000 8192.0000 8192.0000 
8192.0000 8192.0000 8192.0000 8192.0000 
8192.0000 8192.0000 8192.0000 8192.0000 


Matrix multiplication complete.


## Lab Materials

You can download this notebook using the `File > Download as > Notebook (.ipnyb)` menu item. Source code files can be downloaded from the `File > Download` menu item after opening them.